In [1]:
from src.utils import download_images

In [2]:
import os 
os.getcwd()

'c:\\Users\\manas\\OneDrive\\Documents\\GitHub\\amazonml\\__pycache__'

In [3]:
import pandas as pd
df = pd.read_csv('src/sample_test.csv')

In [4]:
df_86 = df.sample(86)
df_86

,index,image_link,group_id,entity_name
57,57,https://m.media-amazon.com/images/I/613P5cxQH4...,525429,wattage
85,85,https://m.media-amazon.com/images/I/81aZ2ozp1G...,805279,maximum_weight_recommendation
63,63,https://m.media-amazon.com/images/I/61G8bvWOb-...,701880,item_weight
50,50,https://m.media-amazon.com/images/I/51r7U52rh7...,860821,voltage
22,22,https://m.media-amazon.com/images/I/514bY8c4ZI...,752266,width
...,...,...,...,...
65,65,https://m.media-amazon.com/images/I/61O+Yi09ty...,507467,voltage
10,10,https://m.media-amazon.com/images/I/41uwo4PVnu...,640565,depth
2,2,https://m.media-amazon.com/images/I/417NJrPEk+...,939426,maximum_weight_recommendation
55,55,https://m.media-amazon.com/images/I/51y79cwGJF...,200507,height


In [5]:
images_links = df_86['image_link'].to_list()

In [8]:
download_images(images_links, "./images", True)

Exception in thread Thread-14 (_handle_workers):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\manas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\multiprocessing\pool.py", line 522, in _handle_workers
    cls._wait_for_updates(current_sentinels, change_notifier)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\

In [23]:
import cv2
import pytesseract
import numpy as np
import os
import matplotlib.pyplot as plt

# Function to display multiple images in a grid
def display_images(images, title):
    # Adjust the grid size for 86 images (e.g., 18 rows and 5 columns to accommodate)
    fig, axes = plt.subplots(18, 5, figsize=(15, 30))  # 18 rows, 5 columns (90 images capacity)
    fig.suptitle(title, fontsize=16)

    for i, img in enumerate(images):
        if i >= 86:  # Limit to 86 images
            break
        ax = axes[i // 5, i % 5]
        ax.imshow(img, cmap='gray')
        ax.axis('off')
        ax.set_title(f'Image {i+1}')
    
    plt.tight_layout()
    plt.show()

# Initialize an array to store the images
image_paths = [os.path.join('images', filename) for filename in os.listdir('images') if filename.endswith(('.png', '.jpg', '.jpeg'))]

# Read only up to 86 images
original_images = [cv2.imread(path) for path in image_paths[:86] if cv2.imread(path) is not None]



In [24]:
# Grayscale conversion
gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in original_images]



In [25]:
# Apply Otsu's binarization
binary_images = [cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] for img in original_images]

In [26]:
# 4. Noise removal using Median Blur and display
denoised_images = [cv2.medianBlur(binary, 3) for binary in binary_images]

In [27]:
# 5. Dilation and display
kernel = np.ones((1, 1), np.uint8)
dilated_images = [cv2.dilate(denoised, kernel, iterations=1) for denoised in denoised_images]


In [28]:
# 6. Erosion and display
eroded_images = [cv2.erode(dilated, kernel, iterations=1) for dilated in dilated_images]


In [29]:
# 7. Skew Correction (rotation) and display
def correct_skew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated
skew_corrected_images = [correct_skew(eroded) for eroded in eroded_images]

In [30]:
# 8. Resize images (increase size for better OCR performance)
resized_images = [cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC) for image in skew_corrected_images]

In [31]:
# 9. Invert colors for white text on black background (optional, based on image content)
inverted_images = [cv2.bitwise_not(image) for image in resized_images]

In [15]:
# 10. OCR Processing on each preprocessed image
ocr_results = []
for img in inverted_images:
    text = pytesseract.image_to_string(img)
    ocr_results.append(text)

In [16]:
# Output OCR results for each image
for idx, text in enumerate(ocr_results):
    print(f"Text from image {idx+1}:\n{text}\n")

Text from image 1:
20CM

13C



Text from image 2:


Text from image 3:
Bete ane

lige? tage “

ome
Pa
Wey
Se eee ae Wurst amon
See ome pgep at Sb te Mipme
Tike mm re ce. ee


Text from image 4:


Text from image 5:


Text from image 6:
Ideal Bed & Armchair Cover

Neat size foldavay

Reegey tee aie eet
THE Te, BUENO AL Oe

Stcity & fg. Liupyy't



Text from image 7:
ne

ED BVORKSICHIBOUD OITA re



Text from image 8:


Text from image 9:
Ili

50cm



Text from image 10:
44"

6.75 LBS


Text from image 11:
nV

> W929 Lp xeul -,8¢é Ulu Sal

X,

i

wOV

—_>

9ZIS dopjseg


Text from image 12:
6.2 in / 16cm

9.4 in / 24cm


Text from image 13:
Width
1C.M.

Length

9 Meter


Text from image 14:
YUld' Le / W969


Text from image 15:
‘ aN SOutce, an
tute. ¢ ¥ PLCVONt clectncal Overioas. UN verte for
*2V power BOu'ce Cats



Text from image 16:
70 «22110


Text from image 17:
12inches
/30.5cm

6inches
ainches
ee

9.5inches/24.1cm /10.7cm


Text from image 18:
6.5cm

16 cm

Example size
Samsung

In [17]:
import re
import string

def preprocess_ocr_text(ocr_results):
  
    
    # Remove unwanted characters (e.g. punctuation, special characters)
    ocr_results = re.sub(r'[^a-zA-Z0-9\s]', '', ocr_results)

    # Convert to lowercase
    # ocr_results = ocr_results.lower()

    # Remove extra whitespace
    ocr_results = re.sub(r'\s+', ' ', ocr_results)

    # Remove leading and trailing whitespace
    ocr_results = ocr_results.strip()

    return ocr_results

In [18]:
prep_text = []
for text in ocr_results:
  preprocessed_text = preprocess_ocr_text(text)
  prep_text.append(preprocessed_text)


In [19]:
print(prep_text)

['20CM 13C', '', 'Bete ane lige tage ome Pa Wey Se eee ae Wurst amon See ome pgep at Sb te Mipme Tike mm re ce ee', '', '', 'Ideal Bed Armchair Cover Neat size foldavay Reegey tee aie eet THE Te BUENO AL Oe Stcity fg Liupyyt', 'ne ED BVORKSICHIBOUD OITA re', '', 'Ili 50cm', '44 675 LBS', 'nV W929 Lp xeul 8 Ulu Sal X i wOV 9ZIS dopjseg', '62 in 16cm 94 in 24cm', 'Width 1CM Length 9 Meter', 'YUld Le W969', 'aN SOutce an tute PLCVONt clectncal Overioas UN verte for 2V power BOuce Cats', '70 22110', '12inches 305cm 6inches ainches ee 95inches241cm 107cm', '65cm 16 cm Example size Samsung S9 75cm', '', 'Say sul Zp 61Inches seg WNIPSW 97I', '', 'UlGO 2WI8 I', 'Acs er GROWS WITH 32085 YOUR CHILD', '', 'Our FauxLeather Textured Velvet fabrics are durable as they are soft Cust on covers aie renovate and ac te wasabe Lasy losoolcedr a olber surfaces ogg doc hace a ee bes a b i LANGE 75 Maxwell Sofa 6', '', 'Wwer wwe9ss', '', 'ENERGY INFORMATION at H 21 Specd ATOM Electe city Uze airflow Ef ce 14

In [20]:
for text in ocr_results:
    print(f" Text: {text}")

 Text: 20CM

13C


 Text: 
 Text: Bete ane

lige? tage “

ome
Pa
Wey
Se eee ae Wurst amon
See ome pgep at Sb te Mipme
Tike mm re ce. ee

 Text: 
 Text: 
 Text: Ideal Bed & Armchair Cover

Neat size foldavay

Reegey tee aie eet
THE Te, BUENO AL Oe

Stcity & fg. Liupyy't


 Text: ne

ED BVORKSICHIBOUD OITA re


 Text: 
 Text: Ili

50cm


 Text: 44"

6.75 LBS

 Text: nV

> W929 Lp xeul -,8¢é Ulu Sal

X,

i

wOV

—_>

9ZIS dopjseg

 Text: 6.2 in / 16cm

9.4 in / 24cm

 Text: Width
1C.M.

Length

9 Meter

 Text: YUld' Le / W969

 Text: ‘ aN SOutce, an
tute. ¢ ¥ PLCVONt clectncal Overioas. UN verte for
*2V power BOu'ce Cats


 Text: 70 «22110

 Text: 12inches
/30.5cm

6inches
ainches
ee

9.5inches/24.1cm /10.7cm

 Text: 6.5cm

16 cm

Example size
Samsung S9+

7.5cm

 Text: 
 Text: Say sul Z"p

6.1Inches

seg WNIPSW 97I —>

 Text: 
 Text: -

UlGO 2/WI8 I}

 Text: Acs er
GROWS WITH (32085.
YOUR CHILD


 Text: 
 Text: Our Faux-Leather & Textured Velvet
fabrics are durable as they are soft

Cust

In [21]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [22]:
i=0;
for text in ocr_results:
    doc = nlp(text)
    for entity in doc.ents:
        if (entity.label_=="QUANTITY"):
            print(f" Entity: {entity.text}, Label: {entity.label_}")
         

        
    

 Entity: 50cm, Label: QUANTITY
 Entity: 16cm, Label: QUANTITY
 Entity: 24cm, Label: QUANTITY
 Entity: 9.5inches/24.1cm, Label: QUANTITY
 Entity: 6.5cm, Label: QUANTITY
 Entity: 16 cm, Label: QUANTITY
 Entity: 7.5cm, Label: QUANTITY
 Entity: 0.6 Cubic Feet, Label: QUANTITY
 Entity: 68.3mm, Label: QUANTITY
 Entity: 27.8mm, Label: QUANTITY
 Entity: 42.0mm, Label: QUANTITY
 Entity: 11cm, Label: QUANTITY
 Entity: 3.8cm / 1.5inch, Label: QUANTITY
 Entity: 16.9in/43cm, Label: QUANTITY
 Entity: 5.31n/13.5cm, Label: QUANTITY


In [32]:
import csv

# Open a CSV file to write the entity results
with open('ocr_entities.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header for the CSV file
    writer.writerow(['Entity', 'Label'])
    
    # Loop through OCR results and extract entities
    for text in ocr_results:
        doc = nlp(text)
        for entity in doc.ents:
            if entity.label_ == "QUANTITY":
                # Write entity and label to the CSV
                writer.writerow([entity.text, entity.label_])

print("CSV file 'ocr_entities.csv' has been created.")


CSV file 'ocr_entities.csv' has been created.
